In [20]:
import pandas as pd
import bs4
from bs4 import BeautifulSoup  
import csv
import numpy as np
from datetime import datetime
from datetime import timedelta
import math


In [21]:
df = pd.read_csv("Real_Estate_TLV_GOVMAPS.csv",index_col=0)


In [22]:
def drop_col_from_df(df, cols):
    return df.drop(columns=cols, axis=1)

cols_to_drop = ["NEIGHBORHOODID","extent","WKT","POLYGONID","DEALDATE","DEALTYPE","GUSHHELKATAT","PROJECTNAME","STREETCODE","OBJECTID","CITY","CITYID"]



def drop_row_by_value(df, col_name, values):
    # Delete all the rows that contain the value in the parameter and return new dataframe
    if isinstance(values, list):
        df = df.drop(df[df[col_name].isin(values)].index).reset_index(drop=True)
    else:
        df = df.drop(df[df[col_name] == values].index).reset_index(drop=True)
    return df

values_to_drop_from_asset_type = ["nan","מיני פנטהאוז","מגורים","בית בודד","דופלקס","קוטג' חד משפחתי","קוטג' דו משפחתי",'מלונאות','חנות','קרקע למגורים','קבוצת רכישה - קרקע מגורים','None','אופציה','קבוצת רכישה - קרקע מסחרי','חניה','מסחרי + מגורים','דירת נופש','דיור מוגן','קומבינציה','מבנים חקלאיים','תעשיה','מסחרי + משרדים','בניני ציבור','חלוקה/יחוד דירות','מחסנים','אחר','בית אבות','עסק',"קוטג' טורי",'ניוד זכויות בניה','משרד','ללא תיכנון','מלונאות ונופש','משרדים + מגורים','מלאכה',]

df = drop_col_from_df(df,cols_to_drop)
df = drop_row_by_value(df,"ASSETTYPE",values_to_drop_from_asset_type)
df = df.dropna(subset=['centroid','ASSETTYPE','NEIGHBORHOOD','DEALS_DEALAMOUNT','ASSETMETER']).reset_index(drop=True)


(117469, 11)

In [23]:
def convert_date_type(df):
    df.rename(columns = {'DEALDATESTRING':'DEALDATE'}, inplace = True)
    df['DEALDATE'] = pd.to_datetime(df['DEALDATE'], format='%d/%m/%Y')
    df['ASSETMETER'] = df['ASSETMETER'].astype(int)
    df['DEALS_DEALAMOUNT'] = df['DEALS_DEALAMOUNT'].astype(int)
    return df

def drop_rows_by_value_size(df):
    #delete all the rows that contain the value in the parameter and return new dataframe
    return df[(df["ASSETMETER"] >= 30) & (df["ASSETMETER"] <= 350)].reset_index(drop=True)

def drop_rows_by_price(df):
    #delete all the rows that contain the value in the parameter and return new dataframe
    return df[(df["DEALS_DEALAMOUNT"] >= 600000) & (df["DEALS_DEALAMOUNT"] <= 20000000)].reset_index(drop=True)

def drop_rows_year_less_than_2020(df):
    start_date = '01/01/2003'
    end_date = '01/01/2023'
    return df[(df["DEALDATE"] > start_date) & (df["DEALDATE"] < end_date)].reset_index(drop=True)

def drop_duplicates_rows(df):
    return df.drop_duplicates().reset_index(drop=True)
 
df = convert_date_type(df)
df = drop_rows_by_value_size(df)
df = drop_rows_year_less_than_2020(df)
df = drop_duplicates_rows(df)


(58797, 11)

In [24]:
floors = {-1:'מרתף', 
     0:'קרקע', 
     1:'ראשונה', 
     2:'שניה', 
     3:'שלישית', 
     4:'רביעית', 
     5:'חמישית', 
     6:'שישית', 
     7:'שביעית', 
     8:'שמינית', 
     9:'תשיעית', 
     10:'עשירית', 
     11:'אחת עשרה', 
     12:'שתים עשרה', 
     13:'שלוש עשרה', 
     14:'ארבע עשרה', 
     15:'חמש עשרה', 
     16:'שש עשרה', 
     17:'שבע עשרה',  
     18:'שמונה עשרה', 
     19:'תשע עשרה', 
     20:'עשרים', 
     21:'עשרים ואחת', 
     22:'עשרים ושתיים', 
     23:'עשרים ושלוש', 
     24:'עשרים וארבע', 
     25:'עשרים וחמש', 
     26:'עשרים ושש', 
     27:'עשרים ושבע', 
     28:'עשרים ושמונה', 
     29:'עשרים ותשע',
     30:'שלושיים',
     31:'שלושיים ואחת',
     32:'שלושיים ושתיים',
     33:'שלושים ושלוש',
     34:'שלושים וארבע',
     35:'שלושים וחמש',
     36:'שלושים ושש',
     37:'שלושים ושבע',
     38:'שלושים ושמונה',
     39:'שלושים ותשע',
     40:'ארבעים',
    }


def reverse_dict(dictionary):
    new_dict = dict()
    for key, value in dictionary.items():
        new_dict[value] = key
    return new_dict

floor = reverse_dict(floors)


In [25]:
def drop_by_floor(df, floor_dict, floor_col):
    for floor, num in floor_dict.items():
        df.loc[df[floor_col] == floor, floor_col] = num
    df = df[df[floor_col].isin(list(floor_dict.values()))].reset_index(drop=True)
    df[floor_col] = df[floor_col].astype(int)
    return df
            

df = drop_by_floor(df ,floor,'FLOOR')


(50698, 11)

In [26]:
df.to_csv('Real_Estate_TLV_Step_2.csv') 
